In [ ]:
#%%writefile net_bot.py
style_transfer_dir = '/home/pe/Public/Real-time-multi-style-transfer/'
import telebot
from telebot import TeleBot
from telebot import types
import os
import time
import logging
from pymorphy2 import MorphAnalyzer
import requests
import wget
import sys
if style_transfer_dir not in sys.path:
    sys.path.append(style_transfer_dir)
import torch
from torchvision import transforms
from inference.Inferencer import Inferencer
from models.PasticheModel import PasticheModel
from PIL import Image
import os
#from keras_text_to_image.library.dcgan import DCGan
#from keras_text_to_image.library.utility.image_utils import img_from_normalized_img
#from keras_text_to_image.library.utility.img_cap_loader import load_normalized_img_and_its_text
from google_drive_downloader import GoogleDriveDownloader as gdd

#################################################################
#############################SETUP###############################
#################################################################
pic_path = './saved_pics/'

#all_models_path = './pokemons/demo/models/'

from string import punctuation
 
def clean(text):
    if not isinstance(text, str):
        raise TypeError('text должен быть str')
    return ''.join(x for x in text.lower() if x not in punctuation)

#current_model_path = all_models_path + 'model_' + str(len(os.listdir(all_models_path)) - 1)

try:
    os.mkdir(pic_path)
except: 
    pass

morph = MorphAnalyzer() 
logging.basicConfig(filename="botlog.log", level=logging.INFO, filemode='w')
token = '1277130373:AAH7oKdzkHwopPmwVluIIVPFfU_rugTedRM'
bot = TeleBot(token)

#gan = DCGan()
#gan.load_model(current_model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_styles = len(os.listdir(style_transfer_dir + 'styles'))
image_size = 512
model_save_dir = style_transfer_dir + "checkpoints/pastichemodel_1-FINAL.pth"

pastichemodel = PasticheModel(num_styles)

inference = Inferencer(pastichemodel,device,image_size)
inference.load_model_weights(model_save_dir)

logging.info('---started---')
running = False
#################################################################


@bot.message_handler(commands = ['start'])
def startBot(message):
    global running
    logging.info('Preparing to reply to "start" command ' + str(message.chat.id))
    bot.reply_to(message,'Hi, this is an ai tattoo bot! Send me a description of your desirable picture and there will be magic')
    logging.info('Replied to "start" command ' + str(message.chat.id))
    running = True
    return

# @bot.message_handler(func = lambda x : True, content_types = ['text'])
# def new_sketch(message):
#     logging.info('"new" sketch command received ' + str(message.chat.id))
#     bot.send_message(message.chat.id, 'Send me a description of your desirable tattoo')
#     logging.info('sent reply to "new" command successfully, waiting for text // ' + str(message.chat.id))
#     bot.register_next_step_handler(message, getText)
#     return

@bot.message_handler(func = lambda x : True, content_types = ['text'])
def getText(message):
    try:
        os.mkdir(pic_path + str(message.chat.id))
    except:
        pass
    if not running:
        return
    #text = ' '.join(list(map(lambda x : morph.parse(x)[0].normal_form, list(filter(lambda x: len(x) > 0, \
    #                                                                   str(message.text).split(' '))))))
    text = str(message.text)
    logging.info('Received text ' + text + ' // ' + str(message.chat.id))
    text = clean(text).replace(' ', '+')
    opath = ''
    try:
        #print(text)
        resp = requests.get('https://go.mail.ru/search_images?fr=main&frm=main&q=' + text + '&fm=1').text
        resp = resp.split('"orig_url":"')[1].split('"')[0]
        #print(resp)
        opath = pic_path + str(message.chat.id) + '/' + text + '.' + resp.split('/')[-1].split('.')[-1]
        #print(opath)
        wget.download(resp.strip(), out = opath, bar = None)
        logging.info('Saved a pic to ' + opath)
    except Exception as e:
        logging.error('Failed to get image // ' + str(message.chat.id) + ' // ' + str(e))
        bot.send_message(message.chat.id, 'Ooops, we cannot deliver you a picture, try again later please')
        bot.register_next_step_handler(message, getText)
        return
        
    logging.info('Starting stylization //' + str(message.chat.id))
    
    try:
        im = Image.open(opath).convert('RGB')
        infer = inference.eval_image(im, 7, 1.0)
        infer.save(opath)
    except Exception as e:
        logging.error('Failed to stylize image // ' + str(message.chat.id) + ' // ' + str(e))
        bot.send_message(message.chat.id, 'Ooops, we cannot deliver you a picture, try again later please')
        bot.register_next_step_handler(message, getText)
        return
    
    logging.info('Finished stylization //' + str(message.chat.id))
        
    try:
        bot.send_photo(message.chat.id, photo = open(opath, 'rb'))
    except Exception as e:
        logging.error('Failed to send image // ' + str(message.chat.id) + ' // ' + str(e))
        bot.send_message(message.chat.id, 'Ooops, we cannot deliver you a picture, try again later please')
        bot.register_next_step_handler(message, getText)
        return
    logging.info('Sent image to //' + str(message.chat.id))
    return
    
@bot.message_handler(commands = ['notify'])
def broadcast(message):
    
    logging.info('Broadcast received')
    
    if message.from_user.username != 'eles13':
        return
    try:
        bot.send_message(message.chat.id, 'Ready for input')
        bot.register_next_step_handler(message, broadcasting_itself)
    except Exception as e:
        logging.error('Broadcasting start failed // ' + str(e))

def broadcasting_itself(message):
    logging.info('Broadcast starting')
    for idd in os.listdir(pic_path):
        try:
            bot.send_message(int(idd), message.text)
        except Exception as e:
            logging.error('Failed to deliver to ' + idd)
            bot.send_message(message.chat.id, 'Failed to deliver to ' + idd)
            continue
    logging.info('Broadcast finished')
    
@bot.message_handler(commands = ['change'])
def change_model_init(message):
    logging.info('Change model request received')
    bot.send_message(message.chat.id, 'Ready for input, send a shared Google Drive link')
    logging.info('Replied to "change model" request')
    try:
        os.mkdir(all_models_path + 'model_' + str(len(os.listdir(all_models_path))))
        logging.info('Created directory for a new model')
    except:
        logging.error('Failed to create a directory for a new model')
        bot.send_message(message.chat.id, 'Try again, failed to create a directory for a new model')
        return
    bot.register_next_step_handler(message, download_and_change)
    return

def download_and_change(message):
    global gan
    link = str(message.text).split('/')[5]
    logging.info('Received model link ' + link)
    current_model_path = all_models_path + 'model_' + str(len(os.listdir(all_models_path)) - 1)
    logging.info('Starting downloading')
    bot.send_message(message.chat.id, 'Starting downloading')
    try:
        gdd.download_file_from_google_drive(file_id=link,
                                        dest_path=current_model_path + '/archive.zip',
                                        unzip=True)
    except Exception as e:
        bot.send_message(message.chat.id, 'Failed to download, send link again, exception ' + str(e))
        bot.register_next_step_handler(download_and_change)
        logging.error('Failed to download, exception ' + str(e))
        return
    logging.info('Finished downloading')
    gan = DCGan()
    gan.load_model(current_model_path)
    logging.info('Model changed')
    bot.send_message(message.chat.id, 'Successfully changed model to iteration ' + str(len(os.listdir(all_models_path)) - 1))
    return
    

bot.polling(none_stop=True, interval = 0)

